# 1. 모듈 불러오기

In [12]:
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse import issparse
from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.preprocessing import MultiLabelBinarizer
import random
import math
from numpy import linalg as la

# 2. 데이터 불러오기

In [13]:
sions_train = np.array([[0.7,0.2,0.1,0.5],[0.2,0.1,0.6,0.4],[0.1,0.7,0.2,0.5],[0.9,0.2,0.5,0.2]])
sions_train_target = np.array([[1,0,0,1,0],[0,1,0,0,1],[0,1,0,0,1],[1,0,1,0,1]])

In [14]:
sions_test = np.array([[0.1,0.2,0.4,0.9],[0.7,0.1,0.1,0.2],[0.1,0.8,0.1,0.1]])

# 3. 클래스 생성하기 - 학습, 테스트

In [15]:
class BPMLL:
    
    def __init__(self, neural=0.2, epoch=20, weight_decay=0.00001, regularization=0.1, print_procedure=False):
        self.features = 0
        self.classes = 0
        self.samples = 0
        self.neural_num = 0
        self.learn_rate = 0.05
        self.neural_percent = neural
        self.epoch = epoch
        self.weightsDecayCost = weight_decay
        self.regularization = regularization
        self.error_small_change = 0.00001
        self.final_error = 0
        self.dataset = []
        self.threshold = None
        self.wsj_matrix = []
        self.vhs_matrix = []
        self.bias_b = []
        self.bias_a = []
        self.print_procedure = print_procedure
        self.trained = False

    def fit(self, x, y):
        self.features = x.shape[1]  
        self.classes = y.shape[1]  
        self.dataset = self.prepare_data(x, y)
        self.samples = len(self.dataset)
        self.neural_num = int(self.features * self.neural_percent)   
        self.wsj_matrix = np.random.random_sample((self.neural_num, self.classes)) - 0.5  
        self.vhs_matrix = np.random.random_sample((self.features, self.neural_num)) - 0.5  
        self.bias_b = np.ones((1, self.classes))  
        self.bias_a = np.ones((1, self.neural_num))  
        self.iterate_training()
        self.trained = True
        return self

    def prepare_data(self, x, y):
        dataset = [] 
        for i in range(x.shape[0]):
            dataset.append(TrainPair(x[i], y[i]))
        return dataset

    def iterate_training(self):
        prev_error = self.global_error()
        for ep in range(self.epoch):
            if self.print_procedure:
                print("학습된 데이터의 epoch는 " + str(ep))
            for i in range(self.samples):
                self.fit_once(i)
            error = self.global_error()
            diff = prev_error - error
            if diff <= self.error_small_change * prev_error:
                self.build_threshhold()
                self.final_error = error
                return
            prev_error = error

        self.build_threshhold()
        self.final_error = prev_error
        return

    def fit_once(self, index):
        x = self.dataset[index].attributes
        y = self.dataset[index].labels
        x_vec = np.array([x]).T
        is_label = self.dataset[index].isLabel
        not_label = self.dataset[index].notLabel
        is_label_length = len(is_label)
        not_label_length = len(not_label)
        b, c = self.forward_propagation(x)
        exp_func = math.exp
        dj_sigma = np.zeros((1, self.classes))
        for j in range(self.classes):
            tmp = 0
            if y[j] == 1:
                for l in not_label:
                    tmp += exp_func(-(c[0, j] - c[0, l]))
            else:
                for k in is_label:
                    tmp -= exp_func(-(c[0, k] - c[0, j]))
            dj_sigma[0, j] = tmp
        
        d = (1 / (is_label_length * not_label_length)) * dj_sigma * (1 - np.square(c))
        b_vec = b.T
        d_vec = d.T
        es_sigma_vec = np.dot(self.wsj_matrix, d_vec)
        e_vec = es_sigma_vec * (1 - np.square(b_vec))
        e = e_vec.T
        self.wsj_matrix = (1 - self.weightsDecayCost) * self.wsj_matrix + self.learn_rate * np.dot(b_vec, d)
        self.vhs_matrix = (1 - self.weightsDecayCost) * self.vhs_matrix + self.learn_rate * np.dot(x_vec, e)
        self.bias_b = (1 - self.weightsDecayCost) * self.bias_b + self.learn_rate * d
        self.bias_a = (1 - self.weightsDecayCost) * self.bias_a + self.learn_rate * e

        return

    def forward_propagation(self, x):
        x = np.array([x])
        netb = np.dot(x, self.vhs_matrix) + self.bias_a
        b = tanh(netb)   
        netc = np.dot(b, self.wsj_matrix) + self.bias_b
        c = tanh(netc)
        return b, c

    def global_error(self):
        global_error = 0
        weights_square_sum = np.sum(np.square(self.wsj_matrix)) + np.sum(
                np.square(self.vhs_matrix)) + np.sum(np.square(self.bias_b)) + np.sum(np.square(self.bias_a))
        for i in range(self.samples):
            c = self.forward_propagation(self.dataset[i].attributes)[1]
            yi = self.dataset[i].isLabel
            nyi = self.dataset[i].notLabel
            yi_length = len(yi)
            nyi_length = len(nyi)
            A = np.array([[c[0, l] - c[0, k] for k in yi] for l in nyi])
            global_error += 1 / (yi_length * nyi_length) * np.sum(np.exp(A))
        global_error += self.regularization * weights_square_sum
        return global_error

    def build_threshhold(self):
        model_outputs = []
        ideal_labels = []
        for i in range(self.samples):
            c = self.forward_propagation(self.dataset[i].attributes)[1][0] 
            model_outputs.append(c)
            ideal_labels.append(self.dataset[i].labels)
        self.threshold = ThresholdFunction(model_outputs, ideal_labels)

    def predict(self, x, rank_results=False):
        
        samples, features = x.shape
        result = RankResults()
        for sample_index in range(samples):
            sample_result = []
            c = self.forward_propagation(x[sample_index])[1][0]
            threshold = self.threshold.compute_threshold(c)
            top_label = None
            max_value = 0
            count = 0
            for j in range(self.classes):
                if c[j] >= threshold:
                    count += 1
                    sample_result.append(j)
                if c[j] > max_value:
                    top_label = j
                    max_value = c[j]
            if count == 0:
                sample_result.append(top_label)
            result.add(sample_result, top_label, c)
        if rank_results is False:
            result = result.predictedLabels

        return result

In [16]:
class TrainPair:
    def __init__(self, attributes, labels):
        self.attributes = attributes 
        self.labels = labels 
        self.isLabel = [] 
        self.notLabel = [] 
        for j in range(labels.shape[0]):
            if labels[j] == 1:
                self.isLabel.append(j)
            else:
                self.notLabel.append(j)

In [17]:
class ThresholdFunction:

    def __init__(self, model_output, ideal_labels):
        self.parameters = []
        self.build(model_output, ideal_labels)

    def build(self, model_output, ideal_labels):
        samples = len(ideal_labels)
        labels = len(ideal_labels[0]) 
        threshholds = np.zeros(samples) 

        for sample_index in range(samples): 
            label_value = [float('inf') for i in range(labels)]
            notlabel_value = [float('-inf') for i in range(labels)] 
            for j in range(labels):
                if ideal_labels[sample_index][j] == 1:  
                    label_value[j] = model_output[sample_index][j]   
                else:
                    notlabel_value[j] = model_output[sample_index][j]
       
            label_min = min(label_value)
            notlabel_max = max(notlabel_value)

            if label_min != notlabel_max:
                if label_min == float('inf'):
                    threshholds[sample_index] = notlabel_max + 0.1
                elif notlabel_max == float('-inf'):
                    threshholds[sample_index] = label_min - 0.1
                else:
                    threshholds[sample_index] = (label_min + notlabel_max) / 2
            else:
                threshholds[sample_index] = label_min
        
        model_output = np.concatenate((model_output, np.array([np.ones(samples)]).T), axis=1)
        self.parameters = np.linalg.lstsq(model_output, threshholds)[0]

    def compute_threshold(self, outputs):
        parameter_length = len(self.parameters)
        b_index = parameter_length - 1

        threshold = 0
        for i in range(b_index):
            threshold += outputs[i] * self.parameters[i]
        threshold += self.parameters[b_index]

        return threshold

In [18]:
def tanh(x):
    return 2 / (1 + np.exp(-2 * x)) - 1

In [19]:
class RankResults:
    def __init__(self):
        self.predictedLabels = []
        self.topRankedLabels = []
        self.outputs = []

    def add(self, predict_set, top_label, output):
        self.predictedLabels.append(predict_set)
        self.topRankedLabels.append(top_label)
        self.outputs.append(output)

# 5. 벡터화 및 실시

In [20]:
result = BPMLL(print_procedure=True, neural=0.4, regularization=0, epoch=40000).fit(sions_train, sions_train_target).predict(sions_test, rank_results=True)

학습된 데이터의 epoch는 0
학습된 데이터의 epoch는 1
학습된 데이터의 epoch는 2
학습된 데이터의 epoch는 3
학습된 데이터의 epoch는 4
학습된 데이터의 epoch는 5
학습된 데이터의 epoch는 6
학습된 데이터의 epoch는 7
학습된 데이터의 epoch는 8
학습된 데이터의 epoch는 9
학습된 데이터의 epoch는 10
학습된 데이터의 epoch는 11
학습된 데이터의 epoch는 12
학습된 데이터의 epoch는 13
학습된 데이터의 epoch는 14
학습된 데이터의 epoch는 15
학습된 데이터의 epoch는 16
학습된 데이터의 epoch는 17
학습된 데이터의 epoch는 18
학습된 데이터의 epoch는 19
학습된 데이터의 epoch는 20
학습된 데이터의 epoch는 21
학습된 데이터의 epoch는 22
학습된 데이터의 epoch는 23
학습된 데이터의 epoch는 24
학습된 데이터의 epoch는 25
학습된 데이터의 epoch는 26
학습된 데이터의 epoch는 27
학습된 데이터의 epoch는 28
학습된 데이터의 epoch는 29
학습된 데이터의 epoch는 30
학습된 데이터의 epoch는 31
학습된 데이터의 epoch는 32
학습된 데이터의 epoch는 33
학습된 데이터의 epoch는 34
학습된 데이터의 epoch는 35
학습된 데이터의 epoch는 36
학습된 데이터의 epoch는 37
학습된 데이터의 epoch는 38
학습된 데이터의 epoch는 39
학습된 데이터의 epoch는 40
학습된 데이터의 epoch는 41
학습된 데이터의 epoch는 42
학습된 데이터의 epoch는 43
학습된 데이터의 epoch는 44
학습된 데이터의 epoch는 45
학습된 데이터의 epoch는 46
학습된 데이터의 epoch는 47
학습된 데이터의 epoch는 48
학습된 데이터의 epoch는 49
학습된 데이터의 epoch는 50
학습된 데이터의 epoch는 51
학습된 데이터의 epoch는 52
학습된

In [21]:
result.outputs

[array([-0.92372237,  0.93761377, -0.64747507, -0.99998335,  0.99973787]),
 array([ 0.99967897, -0.99995778, -0.26786982,  0.93710382, -0.8942125 ]),
 array([-0.96956326,  0.97917602, -0.6696073 , -0.99999494,  0.9998967 ])]

In [22]:
result.topRankedLabels

[4, 0, 4]